In [1]:
from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, CommandHandler, filters, ContextTypes
import requests
import tempfile
import os
import json
import nest_asyncio
nest_asyncio.apply()

TELEGRAM_TOKEN = '8185298084:AAGS56_XjTPeQZqB8zNCIGS6wE5w93wW_mk'
FASTAPI_URL = 'http://127.0.0.1:8000/struk-batch'

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Kirim 1 atau lebih foto struk. Aku akan membalas dengan JSON hasil OCR & pencocokan.")

async def handle_photos(update: Update, context: ContextTypes.DEFAULT_TYPE):
    media_group_id = update.message.media_group_id
    chat_data = context.chat_data

    # Untuk multiple photo (media group), simpan sementara
    if media_group_id:
        photo_files = chat_data.setdefault(media_group_id, [])
    else:
        photo_files = []

    for photo in update.message.photo:
        file = await photo.get_file()
        temp = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
        await file.download_to_drive(custom_path=temp.name)
        photo_files.append(temp.name)

    if not media_group_id or len(photo_files) == len(update.message.photo):
        files = []
        try:
            # Buka semua file untuk POST
            for fpath in photo_files:
                f = open(fpath, 'rb')
                files.append(('files', (os.path.basename(fpath), f, 'image/jpeg')))

            # Kirim ke FastAPI
            response = requests.post(FASTAPI_URL, files=files)
            if response.status_code == 200:
                hasil = response.json()
                hasil_json = json.dumps(hasil, indent=2, ensure_ascii=False)
                await update.message.reply_text(f"```json\n{hasil_json}\n```", parse_mode='Markdown')
            else:
                await update.message.reply_text(f"❌ Gagal memproses, status code: {response.status_code}")

        finally:
            # Tutup semua file
            for _, file_tuple in files:
                file_tuple[1].close()
            # Hapus semua file temp
            for fpath in photo_files:
                os.unlink(fpath)
            if media_group_id:
                chat_data.pop(media_group_id, None)

async def echo_nonphoto(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Silakan kirim foto struk untuk diproses!")

if __name__ == '__main__':
    app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.PHOTO, handle_photos))
    app.add_handler(MessageHandler(~filters.PHOTO, echo_nonphoto))

    print("Bot Telegram berjalan...")
    app.run_polling()


Bot Telegram berjalan...


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "d:\pkl\env\Lib\site-packages\telegram\ext\_application.py", line 1195, in process_update
    await coroutine
  File "d:\pkl\env\Lib\site-packages\telegram\ext\_basehandler.py", line 153, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\naeko\AppData\Local\Temp\ipykernel_17180\311579995.py", line 55, in handle_photos
    os.unlink(fpath)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\naeko\\AppData\\Local\\Temp\\tmp0wsf7tn5.jpg'
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "d:\pkl\env\Lib\site-packages\telegram\ext\_application.py", line 1195, in process_update
    await coroutine
  File "d:\pkl\env\Lib\site-packages\telegram\ext\_basehandler.py", line 153, in handle_update
    return awai

RuntimeError: Cannot close a running event loop

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from telegram import Update, File
from telegram.ext import ApplicationBuilder, MessageHandler, CommandHandler, filters, ContextTypes
import aiohttp
import os
import tempfile
import json
import asyncio
from collections import defaultdict

BOT_TOKEN = "8185298084:AAGS56_XjTPeQZqB8zNCIGS6wE5w93wW_mk"
FASTAPI_ENDPOINT = "http://127.0.0.1:8000/struk-batch"

# Menyimpan gambar per media_group_id sementara
media_group_data = defaultdict(list)
MEDIA_GROUP_TIMEOUT = 3  # detik timeout antar pesan dalam satu media group

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Halo! 👋\nKirim album (media group) gambar struk untuk diproses batch ya!")

async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    media_group_id = update.message.media_group_id
    if media_group_id:
        photo_file = await update.message.photo[-1].get_file()
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp:
            await photo_file.download_to_drive(tmp.name)
            media_group_data[media_group_id].append(tmp.name)

        # Jadwalkan pengolahan setelah timeout
        async def process_group():
            await asyncio.sleep(MEDIA_GROUP_TIMEOUT)
            if media_group_id in media_group_data:
                paths = media_group_data.pop(media_group_id)
                await process_images_batch(update, paths)

        asyncio.create_task(process_group())
    else:
        # Jika bukan album, langsung proses 1 gambar saja
        await process_images_batch(update, [await save_temp_image(update.message.photo[-1])])

async def save_temp_image(photo: File):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp:
        file = await photo.get_file()
        await file.download_to_drive(tmp.name)
        return tmp.name

async def process_images_batch(update: Update, paths):
    try:
        async with aiohttp.ClientSession() as session:
            data = aiohttp.FormData()
            for idx, path in enumerate(paths):
                data.add_field("files", open(path, "rb"), filename=f"struk_{idx}.jpg", content_type="image/jpeg")

            async with session.post(FASTAPI_ENDPOINT, data=data) as resp:
                if resp.status == 200:
                    result = await resp.json()
                    pretty_json = json.dumps(result, indent=2, ensure_ascii=False)
                    await update.message.reply_text(f"```json\n{pretty_json}\n```", parse_mode="Markdown")
                else:
                    error = await resp.text()
                    await update.message.reply_text(f"❌ Gagal memproses struk:\n{error}")
    except Exception as e:
        await update.message.reply_text(f"❌ Terjadi kesalahan: {str(e)}")
    finally:
        for path in paths:
            if os.path.exists(path):
                os.remove(path)

async def run_bot():
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    print("🤖 Bot Telegram aktif...")
    await app.run_polling()

await run_bot()


🤖 Bot Telegram aktif...
